In [5]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Sat Nov 12 01:09:17 2016

@author: stephen
"""

from __future__ import print_function
  
from keras.models import Model
from keras.layers import Input, Dense, merge, Activation
from keras.utils import np_utils
import numpy as np
import pandas as pd
import keras 
from keras.callbacks import ReduceLROnPlateau

np.random.seed(813306)
 
def build_resnet(input_shape, n_feature_maps, nb_classes):
    print ('build conv_x')
    x = Input(shape=(input_shape))
    conv_x = keras.layers.normalization.BatchNormalization()(x)
    conv_x = keras.layers.Conv2D(n_feature_maps, 8, 1, border_mode='same')(conv_x)
    conv_x = keras.layers.normalization.BatchNormalization()(conv_x)
    conv_x = Activation('relu')(conv_x)
     
    print ('build conv_y')
    conv_y = keras.layers.Conv2D(n_feature_maps, 5, 1, border_mode='same')(conv_x)
    conv_y = keras.layers.normalization.BatchNormalization()(conv_y)
    conv_y = Activation('relu')(conv_y)
     
    print ('build conv_z')
    conv_z = keras.layers.Conv2D(n_feature_maps, 3, 1, border_mode='same')(conv_y)
    conv_z = keras.layers.normalization.BatchNormalization()(conv_z)
     
    is_expand_channels = not (input_shape[-1] == n_feature_maps)
    if is_expand_channels:
        shortcut_y = keras.layers.Conv2D(n_feature_maps, 1, 1,border_mode='same')(x)
        shortcut_y = keras.layers.normalization.BatchNormalization()(shortcut_y)
    else:
        shortcut_y = keras.layers.normalization.BatchNormalization()(x)
    print ('Merging skip connection')
    y = merge([shortcut_y, conv_z], mode='sum')
    y = Activation('relu')(y)
     
    print ('build conv_x')
    x1 = y
    conv_x = keras.layers.Conv2D(n_feature_maps*2, 8, 1, border_mode='same')(x1)
    conv_x = keras.layers.normalization.BatchNormalization()(conv_x)
    conv_x = Activation('relu')(conv_x)
     
    print ('build conv_y')
    conv_y = keras.layers.Conv2D(n_feature_maps*2, 5, 1, border_mode='same')(conv_x)
    conv_y = keras.layers.normalization.BatchNormalization()(conv_y)
    conv_y = Activation('relu')(conv_y)
     
    print ('build conv_z')
    conv_z = keras.layers.Conv2D(n_feature_maps*2, 3, 1, border_mode='same')(conv_y)
    conv_z = keras.layers.normalization.BatchNormalization()(conv_z)
     
    is_expand_channels = not (input_shape[-1] == n_feature_maps*2)
    if is_expand_channels:
        shortcut_y = keras.layers.Conv2D(n_feature_maps*2, 1, 1,border_mode='same')(x1)
        shortcut_y = keras.layers.normalization.BatchNormalization()(shortcut_y)
    else:
        shortcut_y = keras.layers.normalization.BatchNormalization()(x1)
    print ('Merging skip connection')
    y = merge([shortcut_y, conv_z], mode='sum')
    y = Activation('relu')(y)
     
    print ('build conv_x')
    x1 = y
    conv_x = keras.layers.Conv2D(n_feature_maps*2, 8, 1, border_mode='same')(x1)
    conv_x = keras.layers.normalization.BatchNormalization()(conv_x)
    conv_x = Activation('relu')(conv_x)
     
    print ('build conv_y')
    conv_y = keras.layers.Conv2D(n_feature_maps*2, 5, 1, border_mode='same')(conv_x)
    conv_y = keras.layers.normalization.BatchNormalization()(conv_y)
    conv_y = Activation('relu')(conv_y)
     
    print ('build conv_z')
    conv_z = keras.layers.Conv2D(n_feature_maps*2, 3, 1, border_mode='same')(conv_y)
    conv_z = keras.layers.normalization.BatchNormalization()(conv_z)

    is_expand_channels = not (input_shape[-1] == n_feature_maps*2)
    if is_expand_channels:
        shortcut_y = keras.layers.Conv2D(n_feature_maps*2, 1, 1,border_mode='same')(x1)
        shortcut_y = keras.layers.normalization.BatchNormalization()(shortcut_y)
    else:
        shortcut_y = keras.layers.normalization.BatchNormalization()(x1)
    print ('Merging skip connection')
    y = merge([shortcut_y, conv_z], mode='sum')
    y = Activation('relu')(y)
     
    full = keras.layers.pooling.GlobalAveragePooling2D()(y)   
    out = Dense(nb_classes, activation='softmax')(full)
    print ('        -- model was built.')
    return x, out
 
       
def readucr(filename):
    data = np.loadtxt(filename, delimiter = ',')
    Y = data[:,0]
    X = data[:,1:]
    return X, Y
   
nb_epochs = 1500
foder = u"E:/Jupyter File/UCR_TS_Archive_2015/"

# 
#flist = ['Adiac', 'Beef', 'CBF', 'ChlorineConcentration', 'CinC_ECG_torso', 'Coffee', 'Cricket_X', 'Cricket_Y', 'Cricket_Z', 
#'DiatomSizeReduction', 'ECGFiveDays', 'FaceAll', 'FaceFour', 'FacesUCR', '50words', 'FISH', 'Gun_Point', 'Haptics', 
#'InlineSkate', 'ItalyPowerDemand', 'Lighting2', 'Lighting7', 'MALLAT', 'MedicalImages', 'MoteStrain', 'NonInvasiveFatalECG_Thorax1', 
#'NonInvasiveFatalECG_Thorax2', 'OliveOil', 'OSULeaf', 'SonyAIBORobotSurface', 'SonyAIBORobotSurfaceII', 'StarLightCurves', 'SwedishLeaf', 'Symbols', 
#'synthetic_control', 'Trace', 'TwoLeadECG', 'Two_Patterns', 'uWaveGestureLibrary_X', 'uWaveGestureLibrary_Y', 'uWaveGestureLibrary_Z', 'wafer', 'WordsSynonyms', 'yoga']

flist  = ['Adiac']
for each in flist:
    fname = each
    x_train, y_train = readucr(foder+fname+'/'+fname+'_TRAIN')
    x_test, y_test = readucr(foder+fname+'/'+fname+'_TEST')
    nb_classes = len(np.unique(y_test))
    batch_size = min(x_train.shape[0]/10, 16)
     
    y_train = (y_train - y_train.min())/(y_train.max()-y_train.min())*(nb_classes-1)
    y_test = (y_test - y_test.min())/(y_test.max()-y_test.min())*(nb_classes-1)
     
     
    Y_train = np_utils.to_categorical(y_train, nb_classes)
    Y_test = np_utils.to_categorical(y_test, nb_classes)
     
    x_train_mean = x_train.mean()
    x_train_std = x_train.std()
    x_train = (x_train - x_train_mean)/(x_train_std)
      
    x_test = (x_test - x_train_mean)/(x_train_std)
    x_train = x_train.reshape(x_train.shape + (1,1,))
    x_test = x_test.reshape(x_test.shape + (1,1,))
     
     
    x , y = build_resnet(x_train.shape[1:], 64, nb_classes)
    model = Model(input=x, output=y)
    optimizer = keras.optimizers.Adam()
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
      
    reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5,
                      patience=50, min_lr=0.0001) 
    hist = model.fit(x_train, Y_train, batch_size=batch_size, nb_epoch=nb_epochs,
              verbose=1, validation_data=(x_test, Y_test), callbacks = [reduce_lr])
    log = pd.DataFrame(hist.history)
    print(log.loc[log['loss'].idxmin]['loss'], log.loc[log['loss'].idxmin]['val_acc'])

build conv_x


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (8, 1), padding="same")`


build conv_y


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 1), padding="same")`


build conv_z


...

390/390 [==============================] - 7s - loss: 3.2628e-04 - acc: 1.0000 - val_loss: 0.8510 - val_acc: 0.8210
Epoch 1201/1500
390/390 [==============================] - 7s - loss: 2.1884e-04 - acc: 1.0000 - val_loss: 0.8448 - val_acc: 0.8235
Epoch 1202/1500
390/390 [==============================] - 7s - loss: 1.6929e-04 - acc: 1.0000 - val_loss: 0.8424 - val_acc: 0.8235
Epoch 1203/1500
390/390 [==============================] - 7s - loss: 1.7040e-04 - acc: 1.0000 - val_loss: 0.8434 - val_acc: 0.8235
Epoch 1204/1500
390/390 [==============================] - 7s - loss: 2.0938e-04 - acc: 1.0000 - val_loss: 0.8395 - val_acc: 0.8261
Epoch 1205/1500
390/390 [==============================] - 7s - loss: 2.1498e-04 - acc: 1.0000 - val_loss: 0.8323 - val_acc: 0.8184
Epoch 1206/1500
390/390 [==============================] - 7s - loss: 1.8219e-04 - acc: 1.0000 - val_loss: 0.8366 - val_acc: 0.8210
Epoch 1207/1500
390/390 [==============================] - 7s - loss: 1.5666e-04 - acc: 1.00

390/390 [==============================] - 7s - loss: 2.2760e-04 - acc: 1.0000 - val_loss: 0.8309 - val_acc: 0.8031
Epoch 1263/1500
390/390 [==============================] - 7s - loss: 2.3831e-04 - acc: 1.0000 - val_loss: 0.8066 - val_acc: 0.8184
Epoch 1264/1500
390/390 [==============================] - 7s - loss: 1.7392e-04 - acc: 1.0000 - val_loss: 0.8049 - val_acc: 0.8184
Epoch 1265/1500
390/390 [==============================] - 7s - loss: 6.6894e-04 - acc: 1.0000 - val_loss: 1.0178 - val_acc: 0.7494
Epoch 1266/1500
390/390 [==============================] - 7s - loss: 5.6354e-04 - acc: 1.0000 - val_loss: 0.9514 - val_acc: 0.8005
Epoch 1267/1500
390/390 [==============================] - 7s - loss: 2.6484e-04 - acc: 1.0000 - val_loss: 0.8790 - val_acc: 0.8184
Epoch 1268/1500
390/390 [==============================] - 7s - loss: 2.1381e-04 - acc: 1.0000 - val_loss: 0.8670 - val_acc: 0.8286
Epoch 1269/1500
390/390 [==============================] - 7s - loss: 2.1403e-04 - acc: 1.00

390/390 [==============================] - 7s - loss: 1.7097e-04 - acc: 1.0000 - val_loss: 0.8768 - val_acc: 0.8133
Epoch 1325/1500
390/390 [==============================] - 7s - loss: 1.1775e-04 - acc: 1.0000 - val_loss: 0.8708 - val_acc: 0.8159
Epoch 1326/1500
390/390 [==============================] - 7s - loss: 2.5864e-04 - acc: 1.0000 - val_loss: 0.8869 - val_acc: 0.8210
Epoch 1327/1500
390/390 [==============================] - 7s - loss: 1.3022e-04 - acc: 1.0000 - val_loss: 0.8818 - val_acc: 0.8210
Epoch 1328/1500
390/390 [==============================] - 7s - loss: 9.8902e-05 - acc: 1.0000 - val_loss: 0.9087 - val_acc: 0.8184
Epoch 1329/1500
390/390 [==============================] - 7s - loss: 8.6061e-05 - acc: 1.0000 - val_loss: 0.8917 - val_acc: 0.8210
Epoch 1330/1500
390/390 [==============================] - 7s - loss: 1.4422e-04 - acc: 1.0000 - val_loss: 0.8852 - val_acc: 0.8235
Epoch 1331/1500
390/390 [==============================] - 7s - loss: 1.3416e-04 - acc: 1.00

390/390 [==============================] - 7s - loss: 1.2400e-04 - acc: 1.0000 - val_loss: 0.8811 - val_acc: 0.8286
Epoch 1387/1500
390/390 [==============================] - 7s - loss: 1.2479e-04 - acc: 1.0000 - val_loss: 0.8804 - val_acc: 0.8286
Epoch 1388/1500
390/390 [==============================] - 7s - loss: 1.5037e-04 - acc: 1.0000 - val_loss: 0.8785 - val_acc: 0.8261
Epoch 1389/1500
390/390 [==============================] - 7s - loss: 8.8389e-05 - acc: 1.0000 - val_loss: 0.8762 - val_acc: 0.8261
Epoch 1390/1500
390/390 [==============================] - 7s - loss: 1.6242e-04 - acc: 1.0000 - val_loss: 0.8758 - val_acc: 0.8261
Epoch 1391/1500
390/390 [==============================] - 7s - loss: 4.4385e-04 - acc: 1.0000 - val_loss: 0.8685 - val_acc: 0.8235
Epoch 1392/1500
390/390 [==============================] - 7s - loss: 1.4315e-04 - acc: 1.0000 - val_loss: 0.8746 - val_acc: 0.8210
Epoch 1393/1500
390/390 [==============================] - 7s - loss: 2.2457e-04 - acc: 1.00

390/390 [==============================] - 7s - loss: 1.8408e-04 - acc: 1.0000 - val_loss: 0.8903 - val_acc: 0.8338
Epoch 1449/1500
390/390 [==============================] - 7s - loss: 2.0204e-04 - acc: 1.0000 - val_loss: 0.8860 - val_acc: 0.8338
Epoch 1450/1500
390/390 [==============================] - 7s - loss: 1.5303e-04 - acc: 1.0000 - val_loss: 0.9485 - val_acc: 0.8312
Epoch 1451/1500
390/390 [==============================] - 7s - loss: 1.5119e-04 - acc: 1.0000 - val_loss: 0.9401 - val_acc: 0.8338
Epoch 1452/1500
390/390 [==============================] - 7s - loss: 1.4726e-04 - acc: 1.0000 - val_loss: 0.9219 - val_acc: 0.8338
Epoch 1453/1500
390/390 [==============================] - 7s - loss: 2.5386e-04 - acc: 1.0000 - val_loss: 0.9012 - val_acc: 0.8286
Epoch 1454/1500
390/390 [==============================] - 7s - loss: 1.0569e-04 - acc: 1.0000 - val_loss: 0.9022 - val_acc: 0.8107
Epoch 1455/1500
390/390 [==============================] - 7s - loss: 1.1635e-04 - acc: 1.00

In [6]:
from keras.models import load_model
model.save('model/model_ResNet.h5')  # creates a HDF5 file 'my_model.h5'